In [ ]:
import requests
import pandas as pd
from functools import reduce

pd.options.display.float_format = '{:.0f}'.format


In [ ]:
# Last n runs to review
params = {"n": 50} # you can adjust this

# Get run parameter data for last n runs
url = "http://localhost:8000/database/last_n_runs/tbl_model_run_params"
model_param_data_json = requests.get(url, params=params)
model_param_data = model_param_data_json.json() 
model_param_data = pd.DataFrame(model_param_data)
model_param_data = model_param_data[["cflt_run_id", "cint_nr_agents"]].drop_duplicates()

# Get local state data for last n runs
url = "http://localhost:8000/database/last_n_runs/tbl_local_state"
local_data_json = requests.get(url, params=params)
local_data = local_data_json.json() 
local_data = pd.DataFrame(local_data)
               
# Get stop information for each run to know how many episodes we should expect to be recorded
url = "http://localhost:8000/database/last_n_runs/tbl_model_runs"
run_data_json = requests.get(url, params=params)
run_data = run_data_json.json() 
run_data = pd.DataFrame(run_data)
run_data_stop = run_data[run_data["cstr_model_status"].isin(["stopped", "completed", "ended in error"])][["cflt_run_id","cstr_model_status"]]
run_data_pauses = run_data[run_data["cstr_model_status"]=="paused"][["cflt_run_id","cstr_model_status"]]
run_data_pauses = run_data_pauses.groupby("cflt_run_id").agg(pause_count=("cstr_model_status", "count")).reset_index()


In [ ]:
# Get the total number of episodes recorded and the max episode count. 
# If the number of episodes recroded is less than the episode count, that indicates that model output wasn't properly committed.
# NOTE: When everything is working as expected, the episode_count value will always be greater than max_episode since the starting episode is 0. 
episode_counts = local_data.groupby('cflt_run_id').agg(
    episode_count=('cint_episode_id', 'nunique'),  
    max_episode=('cint_episode_id', 'max')       
).reset_index()
# NOTE: When everything is working as expected, the episode_count value will always be greater than max_episode since the starting episode is 0. 
# To check for inconsistencies, 1 is substractdd from episode_count to address this. 
episode_counts["Episode Inconsistencies"] = episode_counts["max_episode"] - (episode_counts["episode_count"]-1) 

In [ ]:
to_join = [model_param_data, run_data_stop, run_data_pauses, episode_counts]
review_df = reduce(lambda left, right: pd.merge(left, right, on='cflt_run_id', how='left'), to_join)
review_df.columns = ["Run ID", "# Drones", "Model Termination", "# Pauses", "# Episodes Recorded", "Max Episode Recorded","Episode Inconsistencies"]
display(review_df.sort_values("Run ID"))